# Readme
Run this Notebook with Kernel HEEPS.
This Notebook is partially based on the HEEPS notebooks (HEEPS written by Christian Delacroix) and rewritten by Gilles Otten.

# Importing modules

In [1]:
import numpy as np
import heeps
from astropy.io import fits
import scipy.interpolate
import matplotlib.pyplot as plt
import scipy.signal
import astropy.modeling
from vip_hci.preproc import frame_rotate

from heeps.util.paralang import paralang
import vip_hci

# Generating on and off-axis METIS PSFs with HEEPS
HEEPS is a PSF simulator for high-contrast imaging and coronagraphy and is used in the METIS team to study the impact of various instrumental/telescope effects on the achievable contrast. We will use standard settings to simulate METIS coronagraphic PSFs in N band.

In [2]:
# 1. Create a config dictionary with your simulation parameters in read_config
conf = heeps.config.read_config(verbose=False,band="N2",nframes=100,mode="CVC",call_ScopeSim=False,dit=0.011)
# we set HEEPS to generate PSFs for N2 band and Classical Vortex Coronagraph (CVC).
# keep nframes an even valued number as half of the generated PSFs will be used as a reference.

# 2. Update config parameters. The following parameters
# will be updated to match the selected spectral band and HCI mode:
#   lam, pscale, flux_star, flux_bckg, ls_dRspi, ls_dRint, npupil, ndet,
#   ravc_t, ravc_r

# Please note that the default HEEPS settings do not perfectly match the design parameters of the METIS coronagraphs.
# Similarly, the default AO residual wavefront cube does not perfectly match the AO system of ELT M4 + METIS SCAO (most noticeably the AO control region is square here, but ~round in reality). 
# For this demo we will use the default values.

conf = heeps.config.update_config(saveconf=True, verbose=True, **conf)
print(conf['pscale'],"mas pixel scale",conf['ndet'],"pixels detector window size")
# 3. Load entrance pupil, and create 'wavefront' object
wf = heeps.pupil.pupil(savefits=True, verbose=True, **conf)

# 4.  Create a PSF template for injecting fake exoplanets
psf_off = heeps.wavefront.propagate(wf, onaxis=False, avg=False, savefits=True, verbose=True, **conf);

# 5. Create a cube of on-axis psfs (=star)
psf_on = heeps.wavefront.propagate(wf, onaxis=True, savefits=True, verbose=True, **conf);
print(psf_off[:,:,:].sum()/100) # print the average flux inside the image window.
print(psf_on[:,:,:].sum()/100)



Simulation config: band=N2, mode=CVC
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
   loading vortex back-propagation params
   vc_charge=2, ngrid=1024, diam_norm=1.0838, beam_ratio=0.1072
   npupil=119, pscale=6.7900 mas, lam=1.1330E-05 m
   hfov=1.1 arcsec (-> ndet=325, 17.42 lam/D)

6.79 mas pixel scale 325 detector window size
Entrance pupil: loaded from 'ELT_fullM1.fits'
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
Off-axis PSF:
‾‾‾‾‾‾‾‾‾‾‾‾‾
Load phase screens from 'COMPASS_201810_RandomWind_100screens_meters.fits'
   nscreens=100 (nframes=100, nstep=1)
Create off-axis PSF cube
   apply Lyot stop: circ_ext/int=[False, False], ls_dRext/int/spi=[0.0477, 0.04, 0.0249], ls_misalign=None
   2024-11-28 14:23:56, e2e simulation using 8 cores
   2024-11-28 14:24:29, completed in 33.03 seconds

On-axis PSF:
‾‾‾‾‾‾‾‾‾‾‾‾
Load phase screens from 'COMPASS_201810_RandomWind_100screens_meters.fits'
   nscreens=100 (nframes=100, nstep=1)
Create on-axis PSF cube
   apply Lyot stop: circ_ext/int=[False, False], ls_dRext/int/spi=[0.0477, 0.04, 0.0249],